In [ ]:
from langchain_openai import ChatOpenAI
import httpx

# Create an HTTP client that skips SSL verification (only for hackathon/test environments)
client = httpx.Client(verify=False)
llm = ChatOpenAI(
 base_url="https://genailab.tcs.in",
 model="azure/genailab-maas-gpt-4o",
 api_key="sk-A5iYc_U5ojeN6ZO4I_qvCQ",
 http_client=client
)

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(
 base_url="https://genailab.tcs.in",
 model="azure/genailab-maas-text-embedding-3-large",
 api_key="sk-A5iYc_U5ojeN6ZO4I_qvCQ",
 http_client=client)

In [ ]:
import os
print(os.getcwd())

In [ ]:
import requests
for method in ("get","post","put","delete","head","options","patch"):
    original = getattr(requests,method)

    def insecure_request(*args, _original = original, **kwargs):
        kwargs["verify"] = False
        return _original(*args,**kwargs)
    
    setattr(requests,method,insecure_request)

In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma

loader = PyPDFLoader(r"c:\Users\Genaiblrpiousr13\Downloads\test\checking\files\photos.pdf")
docs = loader.load()
full_text = "\n".join([doc.page_content for doc in docs])

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that chunks documents for semantic search."),
    ("human", "Split the following document into semantically meaningful sections. Return each chunk as a numbered list.\n\n{document}")
])

formatted_prompt = prompt.format_messages(document=full_text)

response = llm.invoke(formatted_prompt)
chunks = response.content.split("\n\n")  # crude split; refine if needed

chunked_docs = [Document(page_content=chunk.strip()) for chunk in chunks if chunk.strip()]

In [ ]:

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vector_store = Chroma.from_documents(
    chunked_docs,
    embedding_model,
    persist_directory="chroma_db"
)
vector_store.persist()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableMap


retriever = vector_store.as_retriever(search_kwargs={"k": 3})
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that answers questions based on retrieved documents."),
    ("human", "Answer the question using the following context:\n\n{context}\n\nQuestion: {question}")
])

# Chain: retrieve → format prompt → invoke LLM
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)
query = "Explain the Camera Handling skills"
response = rag_chain.invoke(query)
print(response.content)
